In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/isobeldaley/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/isobeldaley/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']
Training data created
Instructions f

47/47 [==============================] - 0s 216us/step - loss: 0.0394 - accuracy: 1.0000
Epoch 67/200
47/47 [==============================] - 0s 223us/step - loss: 0.0533 - accuracy: 1.0000
Epoch 68/200
47/47 [==============================] - 0s 229us/step - loss: 0.0625 - accuracy: 1.0000
Epoch 69/200
47/47 [==============================] - 0s 224us/step - loss: 0.0437 - accuracy: 1.0000
Epoch 70/200
47/47 [==============================] - 0s 190us/step - loss: 0.0488 - accuracy: 1.0000
Epoch 71/200
47/47 [==============================] - 0s 216us/step - loss: 0.0381 - accuracy: 0.9787
Epoch 72/200
47/47 [==============================] - 0s 214us/step - loss: 0.0431 - accuracy: 1.0000
Epoch 73/200
47/47 [==============================] - 0s 206us/step - loss: 0.0915 - accuracy: 0.9787
Epoch 74/200
47/47 [==============================] - 0s 222us/step - loss: 0.0555 - accuracy: 1.0000
Epoch 75/200
47/47 [==============================] - 0s 210us/step - loss: 0.0746 - accuracy: 

Epoch 146/200
47/47 [==============================] - 0s 301us/step - loss: 0.0268 - accuracy: 1.0000
Epoch 147/200
47/47 [==============================] - 0s 317us/step - loss: 0.0314 - accuracy: 1.0000
Epoch 148/200
47/47 [==============================] - 0s 353us/step - loss: 0.0261 - accuracy: 1.0000
Epoch 149/200
47/47 [==============================] - 0s 337us/step - loss: 0.0246 - accuracy: 1.0000
Epoch 150/200
47/47 [==============================] - 0s 321us/step - loss: 0.0236 - accuracy: 1.0000
Epoch 151/200
47/47 [==============================] - 0s 284us/step - loss: 0.0264 - accuracy: 1.0000
Epoch 152/200
47/47 [==============================] - 0s 287us/step - loss: 0.0162 - accuracy: 1.0000
Epoch 153/200
47/47 [==============================] - 0s 252us/step - loss: 0.0335 - accuracy: 0.9787
Epoch 154/200
47/47 [==============================] - 0s 287us/step - loss: 0.0038 - accuracy: 1.0000
Epoch 155/200
47/47 [==============================] - 0s 301us/step - lo